In [21]:
class Computer:
    
    def __init__(self, state):
        self.state = state

    def add(self, s, i):
        s[s[i+3]] = s[s[i+1]] + s[s[i+2]]
        return s, i+4, True
    
    def multiply(self, s, i):
        s[s[i+3]] = s[s[i+1]] * s[s[i+2]]
        return s, i+4, True
        
    def kill(self, s, i):
        return s, i, False
                              
    def fail(self):
        print("Error running program!")
        return s, i, False
    
    operations = {
        1: add,
        2: multiply,
        99: kill
    }
    
    def tick(self, state, i):
        op = state[i]
        return self.operations.get(op, self.fail)(self, state, i)
    
    def run_program(self):
        running = True
        i = 0
        while(running):
            self.state, i, running = self.tick(self.state, i)
        return self.state
    